<a href="https://colab.research.google.com/github/taylan-sen/CIS355_FALL05/blob/main/escape_room_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


Use the following code to decrypt messages.

In [6]:
# run this cell to get colab outputs to wrap
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [7]:
# CRYPTO FUNCTIONS TO BE ABLE TO LOAD AN ENCRYPTED API KEY STORED IN THE SHEETd
import cryptography
from getpass import getpass
import base64
from cryptography.fernet import Fernet
import hashlib
import os
from openai import OpenAI

def generate_key(password: str, salt: bytes) -> bytes:
    """Generate a Fernet key from a password and salt."""
    key = hashlib.pbkdf2_hmac('sha256', password.encode(), salt, 100000)
    return base64.urlsafe_b64encode(key)

def encrypt_string(plaintext: str, password: str) -> str:
    """Encrypt a string using Fernet encryption with a salt."""
    # Generate a random salt
    salt = os.urandom(16)
    key = generate_key(password, salt)
    fernet = Fernet(key)

    # Encrypt the plaintext
    ciphertext = fernet.encrypt(plaintext.encode())

    # Return the salt + ciphertext as base64
    return base64.urlsafe_b64encode(salt + ciphertext).decode()

def decrypt_string(encrypted_data: str, password: str) -> str:
    """Decrypt a string using Fernet encryption with a salt."""
    # Decode the base64 string
    data = base64.urlsafe_b64decode(encrypted_data.encode())

    # Extract the salt and ciphertext
    salt, ciphertext = data[:16], data[16:]

    # Generate the key using the extracted salt
    key = generate_key(password, salt)
    fernet = Fernet(key)

    # Decrypt the ciphertext
    plaintext = fernet.decrypt(ciphertext).decode()

    return plaintext

In [ ]:
# USE THIS CELL TO START OPENAI SESSION. USER WILL BE PROMPTED FOR
# PASSWORD TO DECRYPT STORED ENCRYPTED KEY.
encrypted_message = "tm6KG0sX8CWViCLbCj9BCmdBQUFBQUJwTUZEVF90cEVfVlVYc3N1ZlpkR0U3OVVBanRoY3lFTno2VE9KSTBacmtrMDdyTG00c3VvaktwNUtwSmk2MzduNERPOTVkVEN1VHpRNzV5c1ZDRzY3bVV6Z3RHUUZvU3BmLXZ1N3Y3T0R5dkdKQjBQWmhkQ0hhVkp5RVYxbHJOSEdCdFh6T2hSQ21nRXVHcWl1NGR1TkVFTTcwd0VwcFFQQWN4S3VvekMweVkwZjNHMD0="

password = getpass('Enter decryption pass:')

# Decrypt the string
decrypted_api_key = decrypt_string(encrypted_message, password)
print(decrypted_api_key)